In [1]:
import numpy as np
import pandas as pd
import requests
import zipfile
import io

In [2]:
# String needed to access the data in the zipfiles stored at https://f001.backblazeb2.com/file/Backblaze-Hard-Drive-Data

url_15 = 'https://f001.backblazeb2.com/file/Backblaze-Hard-Drive-Data/data_2015.zip'
start_str_15 = '2015/'

url_16_q1 = 'https://f001.backblazeb2.com/file/Backblaze-Hard-Drive-Data/data_Q1_2016.zip'
start_str_16_q1 = 'data_Q1_2016/'

url_16_q2 = 'https://f001.backblazeb2.com/file/Backblaze-Hard-Drive-Data/data_Q2_2016.zip'
start_str_16_q2 = 'data_Q2_2016/'

url_16_q3 = 'https://f001.backblazeb2.com/file/Backblaze-Hard-Drive-Data/data_Q3_2016.zip'
start_str_16_q3 = 'data_Q3_2016/'

url_16_q4 = 'https://f001.backblazeb2.com/file/Backblaze-Hard-Drive-Data/data_Q4_2016.zip'
start_str_16_q4 = ''

url_17_q1 = 'https://f001.backblazeb2.com/file/Backblaze-Hard-Drive-Data/data_Q1_2017.zip'
start_str_17_q1 = ''

url_17_q2 = 'https://f001.backblazeb2.com/file/Backblaze-Hard-Drive-Data/data_Q2_2017.zip'
start_str_17_q2 = ''

url_17_q3 = 'https://f001.backblazeb2.com/file/Backblaze-Hard-Drive-Data/data_Q3_2017.zip'
start_str_17_q3 = ''

url_17_q4 = 'https://f001.backblazeb2.com/file/Backblaze-Hard-Drive-Data/data_Q4_2017.zip'
start_str_17_q4 = 'data_Q4_2017/'

list_url = [[[url_15,start_str_15]], 
            [[url_16_q1,start_str_16_q1], [url_16_q2,start_str_16_q2], [url_16_q3,start_str_16_q3], [url_16_q4,start_str_16_q4]], 
            [[url_17_q1,start_str_17_q1], [url_17_q2,start_str_17_q2], [url_17_q3,start_str_17_q3], [url_17_q4,start_str_17_q4]]
           ]

list_keys = [15, 16, 17]

dict_zipfile = dict(zip(list_keys,list_url))

#  What hard-disk model shall we look at?

### The SMART metrics present in the dataset are model-dependent, thus the various models have to be treated differently. In this project we will focus on a single model. 

With this script we aim to capture the most convenient model to focus on. We deduce that the model ST4000DM000 is the most used throughout 2015, 2016, and 2017 with 37600 differend hard disks of which about 7% have failed.   

### We load the datasets and extract the relevant columns and save them in ModelsDetail_15, ModelsDetail_16, and ModelsDetail_17. The relevant columns are :

date : The time stamp of the observations.

serial_number : Uniquely identifies a hard-disk. It is used to determine the number of distinct entries.

model : Identifies the model of the hard-disk. 

failure : A value in {0,1}. When a 1 is present, the hard disk has failed on the specific date and is removed from the dataset.


In [3]:
def read_entries_and_failures( url, start_string = ''):
    '''This function reads the zip file located at url, open it, read the columns needed to capture the ratio failures/entries'''
    ListDF = []
    
    r = requests.get(url)
    z = zipfile.ZipFile(io.BytesIO(r.content))

    files = [name for name in z.namelist() if (name.endswith('.csv')) & (name.startswith(start_string))]

    for file in files : 
        data = pd.read_csv(z.open(file))
        data2 = data[['date','serial_number', 'model', 'failure']]
        ListDF.append(data2)
    
    df = pd.concat(ListDF, ignore_index = True)
    
    return df
    
    
    
def per_model_entries_and_failures (df) :   
    '''Return dataframe with number of failures and unique serial numbers'''
    FailuresPerModel = df.groupby('model')['failure'].sum()
    EntriesPerModel = df.groupby('model')['serial_number'].unique()

    ModelsDetail = pd.concat([EntriesPerModel,FailuresPerModel],axis=1)
    return ModelsDetail
    
    
def save_yearly_entries_and_failures( dict_url, year ) :
    list_df = []

    for url, start_string in dict_url[year] :

        df = read_entries_and_failures(url, start_string)  
        list_df = list_df + [df]
        
    if len(dict_zipfile[year]) > 1 :
        df = pd.concat(list_df, ignore_index = True)
        
    print('In year ' + str(year) + ' there are ' + str(df.shape[0]) + ' entries.\n')    
        
    return per_model_entries_and_failures(df) 

In [4]:
ModelsDetail_15 = save_yearly_entries_and_failures( dict_zipfile, 15 )
ModelsDetail_16 = save_yearly_entries_and_failures( dict_zipfile, 16 )
ModelsDetail_17 = save_yearly_entries_and_failures( dict_zipfile, 17 )

In year 15 there are 17509251 entries.

In year 16 there are 24471617 entries.

In year 17 there are 30471787 entries.



### For every year we saved the number of failures and an array of unique serial numbers

We combine the information across the three years in a common dataset. Observe that the same hard disk may have been used across different years.

In [5]:
ListDF_raw = [ModelsDetail_15.add_suffix('_15'), ModelsDetail_16.add_suffix('_16'), ModelsDetail_17.add_suffix('_17')]

#df_raw = reduce(lambda x, y: pd.merge(x, y, on = 'model'), ListDF_raw)

DfTot = pd.concat(ListDF_raw, axis = 1) 

DfTot.failure_15 = DfTot.failure_15.fillna(0) 
DfTot.failure_16 = DfTot.failure_16.fillna(0) 
DfTot.failure_17 = DfTot.failure_17.fillna(0) 

DfTot['failure_tot'] = DfTot.failure_15 + DfTot.failure_16 + DfTot.failure_17
del DfTot['failure_15']
del DfTot['failure_16']
del DfTot['failure_17']

DfTot['serial_number_tot'] = [np.hstack((DfTot.loc[mod,'serial_number_15'], DfTot.loc[mod,'serial_number_16'], DfTot.loc[mod,'serial_number_17'])) for mod in DfTot.index]

del DfTot['serial_number_15']
del DfTot['serial_number_16']
del DfTot['serial_number_17']

DfTot['entries_tot'] = [len(set(DfTot.loc[mod,'serial_number_tot'])) for mod in DfTot.index]

del DfTot['serial_number_tot']


For every model we have the number of distinct hard disks and the number of failures observed. 
We look at their ratio and filter the models for which too few hard disks have been sampled.

In [6]:
DfTot['ratio_entry_failures'] = DfTot['failure_tot'].divide(DfTot['entries_tot'], axis = 'rows')
DfTot.sort_values('ratio_entry_failures', inplace=True, ascending =False)

DfTot[DfTot['entries_tot'] > 1000]

,failure_tot,entries_tot,ratio_entry_failures
WDC WD30EFRX,122.0,1261,0.096749
ST3000DM001,106.0,1170,0.090598
ST4000DM000,2590.0,36700,0.070572
ST31500541AS,112.0,1693,0.066155
Hitachi HDS723030ALA640,44.0,1018,0.043222
Hitachi HDS722020ALA330,152.0,4683,0.032458
ST6000DX000,60.0,1938,0.030960
Hitachi HDS5C3030ALA630,96.0,4608,0.020833
Hitachi HDS5C4040ALE630,42.0,2660,0.015789
ST8000DM002,141.0,10029,0.014059


### Each brand has a different way of measuring the SMART metrics. For every model we need to understand which metrics are important and which are not. 

We now select a model and look at the data in the 2015 time frame. This is needed to obtain a mask that will be used to extract only the columns relevant to a specific model. 

Every model is associated with different SMART metrics. Moreover, for every relevant SMART metric we will work with the normalized value (in between 0 and 100/200/260 depending on the metric, the higher the better in tems of performance)

In [7]:
# (2a) Check whether a SMART metric is relevant
def is_relevant(col, thresh_diff = 20, thresh_var = 1) :
    ''' Given a column of the DataFrame, decide whether it is relevant or not'''
    diff = col.max() - col.min()
    var = col.var()
    mincol = col.min()
    if mincol > 99 :
        return False
    if (diff > thresh_diff) | (var > thresh_var):
        return True
    else :
        return False

In [8]:
## The following functions are needed in order to build a mask for any specific model.

# (1) Read the datase associated to a model and a specific mask.
def read_model_and_rel_cols (mod_name, rel_cols, DictUrl, year):
    ''' This function reads the zip file of the year, open it, read the columns associated to a model and 
    the relevant columns '''
    ListDF = []
    
    for url, start_string in DictUrl[year] :
        r = requests.get(url)
        z = zipfile.ZipFile(io.BytesIO(r.content))
        files = [name for name in z.namelist() if (name.endswith('.csv')) & (name.startswith(start_string))]
        for file in files : 
            data = pd.read_csv(z.open(file), parse_dates = [0])
            data2 = data[data['model'] == mod_name]
            data3 = data2.iloc[:,rel_cols]
            ListDF.append(data3)
    
    df = pd.concat(ListDF, ignore_index = True)
    return df



# (2) Return the SMART metrics with no NaN values and relevant variance.
def find_relevant_cols(df, thresh_diff = 20, thresh_var = 1) :
    ''' For the various SMART metrics return True only if the metric is considered by a specific model, i.e. the 
    relative values are not NaN and if the metric is relevant, i.e., the normalized range (max - min) is larger 
    than thresh'''
    non_na_cols = (df.isnull().sum() == 0)
        
    res_bool = []
        
    for col in df.columns :
        if non_na_cols[col] == True :
            res_bool = res_bool + [is_relevant(df[col])]
        else :
            res_bool = res_bool + [False]
    return res_bool
    
    
# (3) Build the mask given the result of RelevantCols    
def build_mask(rel_cols) :
    ''' Esclude all raw columns and all the normalized columns not relevant for a specific model '''
    
    rel_cols_double = [val for val in rel_cols for _ in (0, 1)] 
    
    return [a and b for a, b in zip(rel_cols_double, normalized_cols)]    

In [9]:
# Select the model you want to work with. (ex:'Hitachi HDS722020ALA330', 'ST8000DM002')

#ModelName = 'ST8000DM002' 
ModelName = 'Hitachi HDS722020ALA330' 

## normalized_cols has 'False' in correspondence of a column with a raw value and 'True' of a col with a normalized value
normalized_cols = [True, False] # For each SMART metric the columns are ordered as Normalized,Raw,Normalized,Raw,...

normalized_cols = normalized_cols*45 # ... and there are 45 SMART metrics

# [date, serial number, model, capacity, failure] + SMART metrics
cols = [False, False, False, False, False] + normalized_cols 


In [10]:
df = read_model_and_rel_cols (ModelName, cols, dict_zipfile, 16)

In [11]:
df.describe()

,smart_1_normalized,smart_2_normalized,smart_3_normalized,smart_4_normalized,smart_5_normalized,smart_7_normalized,smart_8_normalized,smart_9_normalized,smart_10_normalized,smart_11_normalized,...,smart_225_normalized,smart_226_normalized,smart_240_normalized,smart_241_normalized,smart_242_normalized,smart_250_normalized,smart_251_normalized,smart_252_normalized,smart_254_normalized,smart_255_normalized
count,930850.000000,930850.000000,930850.000000,930850.0,930850.000000,930850.000000,930850.000000,930850.000000,930850.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
mean,99.800785,103.069698,117.924126,100.0,99.970684,99.999968,101.920021,94.016914,99.998829,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,1.244388,9.505444,5.845312,0.0,0.797564,0.018012,5.975187,0.589591,0.076508,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,47.000000,100.000000,100.000000,100.0,62.000000,84.000000,46.000000,93.000000,95.000000,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,100.000000,100.000000,115.000000,100.0,100.000000,100.000000,100.000000,94.000000,100.000000,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,100.000000,100.000000,116.000000,100.0,100.000000,100.000000,100.000000,94.000000,100.000000,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,100.000000,100.000000,118.000000,100.0,100.000000,100.000000,100.000000,94.000000,100.000000,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
max,100.000000,135.000000,253.000000,100.0,100.000000,100.000000,123.000000,99.000000,100.000000,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Explain why we used range instead of variance, look at smart_1 and smart_2

In [12]:
# Call (2) with the dataset df obtained with the previous call
relevant_cols = find_relevant_cols(df) 

# Call (3) with the relevant columns relevant_cols    
mask = build_mask(relevant_cols)    

sum(mask) # Number of relevant columns

6

### We have the mask relative to the model indicated in ModelName (ex:'Hitachi HDS722020ALA330'). 

We access the online data once again and save only the rows related to ModelName and the columns specified by the mask

In [13]:
# [date, serial number, model, capacity, failure] + SMART metrics
cols = [True, True, False, False, True] + mask 

# Path to the folder in which the datset will be stored
folder_name_model = ModelName.replace(' ', '_')
folder_name = 'Data/' + folder_name_model

In [14]:
# Save the data

for year in [15,16,17] :
    
    df = read_model_and_rel_cols (ModelName, cols, dict_zipfile, year)
    df.to_csv(folder_name + '/Model_' + str(year) + '.csv')
    del df